In [2]:
from pathlib import Path as pt
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [6]:
def generateNGaussian(N):

    gaussfn = lambda n: f"A{n}*(np.exp((-1.0/2.0)*(((x-cen{n})/sigma{n})**2)))+"
    _gfn, _args = "", "x, "
    for i in range(int(N)): 
        _gfn += gaussfn(i)
        _args += f"cen{i}, A{i}, sigma{i}, "
    exec(f"gfn_ = lambda {_args.strip()[:-1]}: {_gfn.strip()[:-1]}")
    
    return locals()["gfn_"]

def computeNGaussian(wn, inten, sigma=5):

    _args = {}
    N = len(wn)
    gfn = generateNGaussian(N)
    i = 0
    for x, y in zip(wn, inten):
        _args[f"cen{i}"] = x
        _args[f"A{i}"] = y
        _args[f"sigma{i}"] = sigma
        i += 1
    full_wn = np.linspace(wn.min()-100, wn.max()+100, 5000)

    full_inten = gfn(full_wn, **_args)
    return full_wn, full_inten 

In [51]:
thz_datlocation = pt(r"D:\SurfDrive\THz_Measurements\CO+\thz\118\EXPORT")
thz_dat_files = list(thz_datlocation.glob("*.dat"))
thz_dat_files

[WindowsPath('D:/SurfDrive/THz_Measurements/CO+/thz/118/EXPORT/20_10_20-3.dat'),
 WindowsPath('D:/SurfDrive/THz_Measurements/CO+/thz/118/EXPORT/20_10_20-4.dat'),
 WindowsPath('D:/SurfDrive/THz_Measurements/CO+/thz/118/EXPORT/21_10_20-2.dat'),
 WindowsPath('D:/SurfDrive/THz_Measurements/CO+/thz/118/EXPORT/21_10_20-3_corrected.dat'),
 WindowsPath('D:/SurfDrive/THz_Measurements/CO+/thz/118/EXPORT/21_10_20-4.dat'),
 WindowsPath('D:/SurfDrive/THz_Measurements/CO+/thz/118/EXPORT/21_10_20-5.dat'),
 WindowsPath('D:/SurfDrive/THz_Measurements/CO+/thz/118/EXPORT/21_10_20-6.dat'),
 WindowsPath('D:/SurfDrive/THz_Measurements/CO+/thz/118/EXPORT/22_10_20-1_corrected.dat'),
 WindowsPath('D:/SurfDrive/THz_Measurements/CO+/thz/118/EXPORT/22_10_20-2.dat'),
 WindowsPath('D:/SurfDrive/THz_Measurements/CO+/thz/118/EXPORT/23_10_20-5.dat'),
 WindowsPath('D:/SurfDrive/THz_Measurements/CO+/thz/118/EXPORT/26_10_20-1.dat'),
 WindowsPath('D:/SurfDrive/THz_Measurements/CO+/thz/118/EXPORT/26_10_20-2.dat'),
 Windows

In [85]:
freq, depletion = np.genfromtxt(thz_dat_files[-1]).T
ind = 80
end = -95
freq, depletion = freq[ind:end], depletion[ind:end]

%matplotlib widget
fig, ax = plt.subplots(figsize=(12, 5), dpi=100)
ax.plot(freq, depletion, ".k", label=f"Experiment")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [187]:
freq_guess = [118.1007, 118.1018, 118.1029]
depletion_guess = [5, 10, 6]
sigma_guess = [1e-4, 1e-3, 1e-4]

init_gueeses = np.array([freq_guess, depletion_guess, sigma_guess]).T.flatten()
N = 3
gfn = generateNGaussian(N)
pop, poc = curve_fit(gfn, freq, depletion, p0=init_gueeses)
perr = np.sqrt(np.diag(poc))
freq_fit = pop[::3]
depletion_fit = pop[1::3]
sigma_fit = pop[2::3]
freq_fit, perr[::3]

(array([118.10067608, 118.10187337, 118.10291072]),
 array([1.83529770e-05, 2.69756984e-05, 4.46651807e-06]))

In [188]:
%matplotlib widget

fig, ax = plt.subplots(figsize=(12, 5), dpi=100)
ax.plot(freq, depletion, ".k", label=f"Experiment")
ax.plot(freq, gfn(freq, *pop), "-k", ms=10, label=f"Fitted")

ax.set(xlabel="Frequency (GHz)", ylabel="Depletion (%)", title="CO$^+$: 0$_{1/2}$ --> 1$_{3/2}$")
ax.stem(freq_fit, depletion_fit, use_line_collection=True, linefmt="k")
ax.legend()
ax.minorticks_on()
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
freq_guess = [117.6912, 117.6918, 117.6929, 117.6934]
depletion_guess = [4.5, 4.5, 4.5, 4]
sigma_guess = [1e-4, 1e-4, 1e-4, 1e-4]

init_gueeses = np.array([freq_guess, depletion_guess, sigma_guess]).T.flatten()
N = 4
gfn = generateNGaussian(N)
pop, poc = curve_fit(gfn, freq, depletion, p0=init_gueeses)
perr = np.sqrt(np.diag(poc))
freq_fit = pop[::3]
depletion_fit = pop[1::3]
sigma_fit = pop[2::3]
freq_fit

C:\Users\aravi\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


array([117.6912, 117.6918, 117.6929, 117.6934])

In [5]:
%matplotlib widget

fig, ax = plt.subplots(figsize=(12, 5), dpi=100)
ax.plot(freq, depletion, ".k", label=f"Experiment")
ax.plot(freq, gfn(freq, *pop), "-k", ms=10, label=f"Fitted")

ax.set(xlabel="Frequency (GHz)", ylabel="Depletion (%)", title="CO$^+$: 0$_{1/2}$ --> 1$_{1/2}$")
ax.stem(freq_fit, depletion_fit, use_line_collection=True, linefmt="k")
ax.legend()
ax.minorticks_on()
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'freq' is not defined